# Terms creation in 2P-Kt

## Goals

- Learn how to create/inspect terms in `2p-kt`
- Understand the representation of lists in logic

## Terms hierarchy in `2p-kt`

Simplified view of the type hierarchy of `Term` in `2p-kt`:
![Type hierarchy of `Term`](http://www.plantuml.com/plantuml/svg/PP11IiOm48NtSufSuB-1Yugu4CI5YlkXdRR1P5ec4qJmy1fYoBHTlk-HyBuPag9eZW7If-ST1QDBAqxRb-V5pHWI0NXMaXb7SkyaqSI7ZPCYgq4VA2QzgUJHrYyvalZHa2TMMiW-UYKC9jxh5kq4oRgQ_1Xt_NxsIGqpiSq4ADlTtS_jpurlRTa1DfypBaxGkewRowMfa-6_xxY8NMax0DH9frzEiAf1gJgh9u5ITTZlSkJr60Fnb6Fu1G00)


## Preliminaries

Let's first import `2p-kt` (`%use` directive only work in Jupyter):

In [1]:
%use 2p-kt@/ktlibs
%use kt-math@/ktlibs

Let's create a parser for converting strings into terms:

In [2]:
val parser = TermParser.withNoOperator()

Let's create a formatter for converting terms into human-readable strings:

In [3]:
val formatter = TermFormatter.prettyVariables()

## Atoms

Atoms are alphanumerics constants.
When
- containing spaces, 
- or beginning with uppercase letters or undescores

atom are represented as wrapped within single/double apices:

In [9]:
val representations = listOf(
    "anAtom", 
    "'an atom with spaces'",
    "\"ACapitalAtom\""
)
representations

[anAtom, 'an atom with spaces', "ACapitalAtom"]

Atoms can be programmatically constructed by means of the `Atom.of` factory method, accepting a `String` *without apices* as input.

Otherwise, atoms can be parsed from a `String` by means of a `TermParser`.

Below, try to construct the atom **programmatically**, out of a representation, in order for it to match the atom parsed from the same representation.

In [10]:
for (repr in representations) {
    val term: Term = TODO("create an ${Atom::class.simpleName} term whose representation is equal to $repr")
    val expected = parser.parseTerm(repr)

    assertEquals(expected, term)
    assertTrue(term is Struct)
    assertTrue(term is Constant)
    assertTrue(term is Atom)
    assertEquals(repr.replace("'", "").replace("\"", ""), term.value)
    assertEquals(repr, formatter.format(term))
}

An operation is not implemented: create an Atom term whose representation is equal to anAtom
kotlin.NotImplementedError: An operation is not implemented: create an Atom term whose representation is equal to anAtom
	at Line_147_jupyter.<init>(Line_147.jupyter.kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:105)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47

## Numbers
### Integers

In [6]:
val integers = listOf(1, 2, 0, -1)

In [7]:
for (int in integers) {
    val term: Term = TODO("create an ${Integer::class.simpleName} term whose value is equal to $int")
    val representation = int.toString()
    val expected = parser.parseTerm(representation)

    assertEquals(expected, term)
    assertTrue(term is Numeric)
    assertTrue(term is Constant)
    assertTrue(term is Integer)
    assertEquals(BigInteger.of(int), term.value)
    assertEquals(representation, formatter.format(term))
}

An operation is not implemented: create an Integer term whose value is equal to 1
kotlin.NotImplementedError: An operation is not implemented: create an Integer term whose value is equal to 1
	at Line_7_jupyter.<init>(Line_7.jupyter.kts:2)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:105)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experi

### Reals

In [8]:
val representations = listOf("1.2", "-3.4", "0.0", kotlin.math.PI.toString())

In [ ]:
for (repr in representations) {
    val term: Term = TODO("Create a ${Real::class.simpleName} term whose representation is equal to $repr")
    val expected = parser.parseTerm(repr)

    assertEquals(expected, term)
    assertTrue(term is Numeric)
    assertTrue(term is Constant)
    assertTrue(term is Real)
    assertEquals(BigDecimal.of(repr), term.value)
    assertEquals(repr, formatter.format(term))
}
    

## Variables

In [ ]:
val names = listOf("A", "B", "_", "_A", "_B", "SomeVariable")

In [ ]:
for (name in names) {
    val term: Term = TODO("Create a ${Var::class.simpleName}iable term whose name is equal to $name")
    val expected = parser.parseTerm(name)

    assertNotEquals(expected, term) // notice this!
    assertTrue(expected.equals(term, useVarCompleteName = false))
    assertTrue(term is Var)
    assertEquals(name, term.name)
    assertEquals(name, formatter.format(term))
    assertTrue(term.completeName.startsWith(name + "_"))
    assertEquals(name == "_", term.isAnonymous)
}

## Structures

In [ ]:
val representation = "person(giovanni, ciatto, 30)"

In [ ]:
val term: Term = TODO("Create a ${Struct::class.simpleName}ured term whose representation is equal to $representation")
val expected = parser.parseTerm(representation)

assertEquals(expected, term)
assertTrue(term is Struct)
assertEquals("person", term.functor)
assertEquals(3, term.arity)
assertEquals(
    listOf(Atom.of("giovanni"), Atom.of("ciatto"), Integer.of(30)),
    term.args
)
assertEquals(term[0], Atom.of("giovanni"))
assertEquals(term[1], Atom.of("ciatto"))
assertEquals(term[2], Integer.of(30))
assertEquals(representation, formatter.format(term))

val newTerm = term.addLast(Var.of("DateOfBirth"))
val newRepresentation = "person(giovanni, ciatto, 30, DateOfBirth)"
val newExpected = parser.parseTerm(newRepresentation)

assertTrue(newExpected.equals(newTerm, useVarCompleteName = false))

## Lists

In [ ]:
val representation = "[1, a, f(x)]"

In [ ]:
val term: Term = TODO("Create a ${List::class.simpleName} term whose representation is equal to $representation")
val expected = parser.parseTerm(representation)

assertEquals(expected, term)
assertTrue(term is Struct)
assertTrue(term is List)
assertTrue(term is Cons)
assertEquals(".", term.functor)
assertEquals(2, term.arity)
assertEquals(term[0], Integer.of(1))
assertEquals(term[1], List.of(Atom.of("a"), Struct.of("f", Atom.of("x"))))
assertEquals(representation, formatter.format(term))

assertEquals(
    term,
    Cons.of(
        head = Integer.of(1),
        tail = Cons.of(
            head = Atom.of("a"),
            tail = Cons.of(
                head = Struct.of("f", Atom.of("x")),
                tail = EmptyList.instance
            )
        )
    )
)